# Make multiwavelength light curves using archival data
***

## Learning Goals    
By the end of this tutorial, you will be able to:
 - automatically load a catalog of sources
 - automatically search NASA and non-NASA resources for light curves
 - store light curves in a Pandas multiindex dataframe
 - plot all light curves on the same plot
 
 
## Introduction:
 - A user has a sample of interesting targets for which they would like to see a plot of available archival light curves.  We start with a small set of changing look AGN from Yang et al., 2018, which are automatically downloaded. Changing look AGN are cases where the broad emission lines appear or disappear (and not just that the flux is variable). We model light curve plots after van Velzen et al. 2021.  We search through a curated list of time-domain NASA holdings as well as non-NASA sources.  HEASARC catalogs used are Fermi and Beppo-Sax, IRSA catalogs used are ZTF and WISE, and MAST catalogs used are Pan-Starrs, TESS, Kepler, and K2.  Non-NASA sources are Gaia and IceCube. This list is hopefully generalized enough to include many types of targets to make this notebook interesting for many types of science.  All of these time-domain archives are searched in an automated fashion using astroquery.
 - Light curve data storage is a tricky problem.  Currently we are using a multi-index Pandas dataframe, but it is not clear that this is the best choice or the best choice for scaling up.  One downside is that we need to manually track the units of flux and time instead of relying on an astropy storage scheme which would be able to do the units worrying for us.  Astropy does not currently have a good option for multi band light curve storage.
 - We intend to explore two possible extensions to this project.  The first is an image extension where we would search the archives for images of these sources.  The second, is exploring a ML classifier for these changing look AGN light curves.
 
## Input:
 - a catalog of CLAGN from the literature

## Output:
 - an archival optical + IR + neutrino light curve
 
## Technical Goals:
 - should be able to run from a clean checkout from github
 - should be able to automatically download all catalogs & images used
 - need to have all photometry in the same physical unit
 - need to have a data structure that is easy to use but holds light curve information (time and units) and is extendable to ML applications
 - need to have a curated list of catalogs to search for photometry that is generalizeable to other input catalogs
 
## Non-standard Imports:
- `astroquery` to interface with archives APIs
- `astropy` to work with coordinates/units and data structures
- `AXS` (astronomy extensions for Spark) to handle large catalog cross matching
- `lightkurve` to search TESSS, Kepler, and K2 archives
- `urllib` to handle archive searches with website interface

## Authors:
IPAC SP team

## Acknowledgements:
Suvi Gezari, Antara Basu-zych,\
MAST, HEASARC, & IRSA Fornax teams



In [1]:
import numpy as np
import time
import pandas as pd
import axs
import os
import sys
import re
import matplotlib.pyplot as plt
import json
import requests

from astroquery.ipac.ned import Ned
from astroquery.heasarc import Heasarc
from astroquery.gaia import Gaia

from astropy.coordinates import SkyCoord
import astropy.units as u
from astropy.table import Table, vstack, hstack, unique
from astropy.io import ascii
from astropy.time import Time


try: # Python 3.x
    from urllib.parse import quote as urlencode
    from urllib.request import urlretrieve
except ImportError:  # Python 2.x
    from urllib import pathname2url as urlencode
    from urllib import urlretrieve

!pip install lightkurve --upgrade
import lightkurve as lk

!pip install acstools
from acstools import acszpt

You should consider upgrading via the '/opt/conda/bin/python3 -m pip install --upgrade pip' command.
You should consider upgrading via the '/opt/conda/bin/python3 -m pip install --upgrade pip' command.


## 1. Define the Sample
 use the following paper to make a sample of CLAGN: https://iopscience.iop.org/article/10.3847/1538-4357/aaca3a 

 This sample can later be switched out to a differen/larger sample of "interesting" targets


In [2]:
#use ADS to find the refcode for this paper (ie., '2018blahblah')
#Then use the NED astroquery interface to load the table from the paper with relevant target info
#This loads the table into an astropy table

CLAGN = Ned.query_refcode('2018ApJ...862..109Y')



In [3]:
#see what is included in this table
CLAGN

No.,Object Name,RA,DEC,Type,Velocity,Redshift,Redshift Flag,Magnitude and Filter,Separation,References,Notes,Photometry Points,Positions,Redshift Points,Diameter Points,Associations
,,degrees,degrees,,km / s,,,,arcmin,,,,,,,
int32,str30,float64,float64,object,float64,float64,object,object,float64,int32,int32,int32,int32,int32,int32,int32
1,WISEA J000107.52-000552.5,0.28136,-0.09789,G,59147.0,0.197292,SPEC,19.7g,--,6,0,34,8,5,4,0
2,WISEA J012648.10-083948.0,21.70037,-8.66335,G,59333.0,0.197913,SPEC,18.6g,--,22,0,62,14,8,8,0
3,2MASS J01595763+0033105,29.99,0.55301,QSO,93546.0,0.312036,SPEC,19.6g,--,63,0,76,17,19,10,0
4,WISEA J015957.63+003310.3,29.99015,0.55288,IrS,93547.0,0.31204,SLS,,--,4,0,12,3,2,0,0
5,WISEA J080347.56+425839.0,120.94815,42.97747,G,125547.0,0.418778,SPEC,21.8g,--,4,0,27,5,3,4,0
6,WISEA J083132.25+364617.0,127.88438,36.77146,G,58463.0,0.19501,SPEC,18.8g,--,17,0,62,9,7,10,0
7,WISEA J084957.78+274728.8,132.49077,27.79139,QSO,89499.0,0.298537,SPEC,19.1g,--,23,0,65,12,11,8,0
8,WISEA J090932.02+474730.7,137.38346,47.79186,G,35059.0,0.116944,SPEC,18.7g,--,19,0,53,8,6,8,1


In [4]:
#Build a list of skycoords from target ra and dec
#This is useful for some codes which understand that these are a set of coordinates
coords_list = [
    SkyCoord(ra, dec, frame='icrs', unit='deg')
    for ra, dec in zip(CLAGN['RA'], CLAGN['DEC'])
]


## 2. Find light curves for these targets in NASA catalogs
- data access concerns:
    - can't ask the archives to search their entire holdings
        - not good enough meta data
        - not clear that the data is all vetted and good enough to include for science
        - all catalogs have differently named columns so how would we know which columns to keep
    - instead work with a curated list of catalogs for each archive
        - focus on general surveys
        - try to ensure that this list is also appropriate for a generalization of this use case to other input catalogs
        - could astroquery.NED be useful in finding a generalized curated list


### 2.1 HEASARC: FERMI & Beppo SAX


In [9]:
mission_list = ['FERMIGTRIG', 'SAXGRBMGRB']
radius = 0.1*u.degree
#for testing
#for ccount in range(1):
    #To get a fermigtrig source
    #coord = SkyCoord('03h41m21.2s -89d00m33.0s', frame='icrs')

    #to get a bepposax source
    #coord = SkyCoord('14h32m00.0s -88d00m00.0s', frame='icrs')

for ccount, coord in enumerate(coords_list):
    #use astroquery to search that position for either a Fermi or Beppo Sax trigger
    for mcount, mission in enumerate(mission_list):
        try:
            results = Heasarc.query_region(coord, mission = mission, radius = radius)#, sortvar = 'SEARCH_OFFSET_')
            
            #really just need to save the one time of the Gamma ray detection
            #time is already in MJD for both catalogs
            if mission == 'FERMIGTRIG':
                time_mjd = results['TRIGGER_TIME']
                
            else:
                time_mjd = float(results['TIME'][0])
                
            #build the storage for this result
            dfsingle = pd.DataFrame(dict(flux=[1.0], err=[0.0], time=[time_mjd], objectid=[ccount + 1], band=[mission])).set_index(["objectid", "band", "time"])

            try:
                df_lc
            except NameError:
                #df_lc doesn't exist (yet)
                df_lc = dfsingle
            else:
                #df_lc exists
                df_lc = pd.concat([df_lc, dfsingle])

        except AttributeError:
            print("no results at that location for ", mission)


#**** These HEASARC searches are returning an attribute error because of an astroquery bug
# bug submitted to astroquery Oct 18, waiting for a fix.
# if that gets fixed, can probably change this cell 

no results at that location for  FERMIGTRIG
no results at that location for  SAXGRBMGRB
no results at that location for  FERMIGTRIG
no results at that location for  SAXGRBMGRB
no results at that location for  FERMIGTRIG
no results at that location for  SAXGRBMGRB
no results at that location for  FERMIGTRIG
no results at that location for  SAXGRBMGRB
no results at that location for  FERMIGTRIG
no results at that location for  SAXGRBMGRB
no results at that location for  FERMIGTRIG
no results at that location for  SAXGRBMGRB
no results at that location for  FERMIGTRIG
no results at that location for  SAXGRBMGRB
no results at that location for  FERMIGTRIG
no results at that location for  SAXGRBMGRB
no results at that location for  FERMIGTRIG
no results at that location for  SAXGRBMGRB
no results at that location for  FERMIGTRIG
no results at that location for  SAXGRBMGRB
no results at that location for  FERMIGTRIG
no results at that location for  SAXGRBMGRB
no results at that location for 

In [10]:
df_lc

,,,flux,err
objectid,band,time,,
1,SAXGRBMGRB,52013.991319,1.0,0.0


### 2.2 IRSA: ZTF

In [ ]:
#python package ztfquery is not a good solution for this because it requires IRSA password
#Instead will construct the URL for an API query
#https://irsa.ipac.caltech.edu/docs/program_interface/ztf_lightcurve_api.html
ztf_radius = 0.000278   #as suggested by Dave Shupe

for ccount, coord in enumerate(coords_list):
    #doesn't take SkyCoord
    ra = CLAGN['RA'][ccount]
    dec = CLAGN['DEC'][ccount]
    #make the string for the URL query
    #ask for all three bands (g, r, i)
    #don't want data that is flagged as unusable by the ZTF pipeline
    urlstr = 'https://irsa.ipac.caltech.edu/cgi-bin/ZTF/nph_light_curves?POS=CIRCLE %f %f %f&BANDNAME=g,r,i&FORMAT=ipac_table&BAD_CATFLAGS_MASK=32768'%(ra, dec,ztf_radius)

    response = requests.get(urlstr)
    if response.ok:
        ztf_lc = ascii.read(response.text, format='ipac')
        #print(count, len(ztf_lc))
        #this could be up to 3 light curves because there are 3 filters
        #need to sort by filtercode 'zg','zr','zi'
        #and store the light curves
    else:
        print(ccount, " There is no ZTF light curve at this position")

### 2.3 IRSA:WISE

- Dave Shupe has made a catalog of NEOWISE light curves of half the sky in a parquet file

- Pandas is not a good option for reading in and working with this catalog because it is so large (2 billion rows?)

- Instead we can use AXS to cross match the CLAGN sample with the NEOWISE catalog to find those rows in neowise which correspond to the CLAGN sample. AXS is a part of spark. 


In [ ]:
#%%time
#could load the neowise light curves into pandas, but would need to severely
# filter the catalog to get it to fit into memory.  Since these targets are all over the sky
# it is not obvious how to filter the catalog

#Here is one way it could work in Pandas if we had a way to filter significantly before matching
#subset = pd.read_parquet('/stage/irsa-data-download10/parquet-work/NEOWISE-R/neowise_lc_half.parquet',
#                    engine='pyarrow', 
#                    filters=[ ('ra', '<', 121) , ('ra', '>', 120) , 
#                            ('dec', '<', 68) , ('dec', '>', -9),
#                            ('cw_w1mpro', '>', 15.0) ])
#
#len(subset)

In [ ]:
#start up SPARK
os.environ['SPARK_CONF_DIR'] = '/home/jkrick/axs_store/conf_alt'

def spark_start(work_dir, database_dir, warehouse_dir):
    from pyspark.sql import SparkSession
    import os
    
    spark = (
            SparkSession.builder
            .appName("spark trial")
            .config("spark.sql.warehouse.dir", warehouse_dir)
            .config('spark.master', "local[20]")
            .config('spark.driver.memory', '64G') # 128
            .config('spark.executor.memory', '30G')
            .config('spark.local.dir', work_dir)
            .config('spark.memory.offHeap.enabled', 'true')
            .config('spark.memory.offHeap.size', '128G') # 256
            .config("spark.sql.execution.arrow.enabled", "true")
            .config("spark.driver.maxResultSize", "60G")
            .config("spark.driver.extraJavaOptions", 
                    f"-Dderby.system.home={database_dir}")
            .config("spark.sql.hive.metastore.sharedPrefixes",
                    "org.apache.derby")
            .enableHiveSupport()
            .getOrCreate()
                    )   

    return spark

spark_session = spark_start(
    "/stage/irsa-staff-jkrick/spark_work",
    "/home/jkrick/axs_store",
    "/stage/irsa-staff-jkrick/sp_axs_warehouse/warehouse")

In [ ]:
#if the catalog we want is not yet available, import it
catalog = axs.AxsCatalog(spark_session)
catlist = catalog.list_table_names()

if 'neowise_lc_half' not in catlist:
    catalog.import_existing_table('neowise_lc_half', 
        '/stage/irsa-data-download10/parquet-work/NEOWISE-R/neowise_lc_half.parquet',
        import_into_spark=True)
    

#now figure out how to get the CLAGN catalog into AXS
#can't go direct from astropy table into AXS, so first need to convert astropy to pandas, then to AXS

if 'axs_clagn' not in catlist:

    pd_CLAGN = CLAGN.to_pandas()

    #then pandas to spark dataframe
    sp_CLAGN = spark_session.createDataFrame(pd_CLAGN)

    #saving below can't handle capital "RA" and "DEC", so need to change that
    #also can't handle column names with spaces in them so need to rename those as well.
    sp_CLAGN2 = sp_CLAGN.withColumnRenamed("RA","ra").withColumnRenamed("DEC","dec").withColumnRenamed("Object Name", "Object_name").withColumnRenamed("Redshift Flag","redshift_flag").withColumnRenamed("Magnitude and Filter", "magnitude_and_filter").withColumnRenamed("Photometry Points","photometry_points").withColumnRenamed("Redshift Points", "redshift_points").withColumnRenamed("Diameter Points","diameter_points")

    #now save spark to AXS
    catalog.save_axs_table(sp_CLAGN2, 'AXS_CLAGN', calculate_zone=True)

In [ ]:
#just confirm that worked:
#catalog = axs.AxsCatalog(spark_session)
#catlist = catalog.list_table_names()
#catlist

In [ ]:
#lazy load in the NEOWISE and CLAGN catalog
neowise_lc_half = catalog.load('neowise_lc_half')
axs_clagn = catalog.load('axs_clagn')

#rename column name which is causing problems
axs_clagn = axs_clagn.withColumnRenamed("No.", "objnum")

In [ ]:
#lazy crossmatch CLAGN with NEOWISE

neowise_CLAGN = neowise_lc_half.crossmatch(axs_clagn, 2*axs.Constants.ONE_ASEC, return_min = True, include_dist_col = True)


In [ ]:
%%time
#lazy evaluation means the cross match won't happen until this cell gets executed
#how many matches did we get?
neowise_CLAGN.count()

In [ ]:
%%time
#result is an AXS table, but want to get it into pandas
#this is slow (18 min for limited set of columns) but is the fastest I have found

#make a smaller version with just the light curve info to save
neowise_lc = neowise_CLAGN.select('objnum','Object_name','w1pmag','w1pmagerr','w2pmag','w2pmagerr', 'mjd')

#convert to Pandas dataframe
pd_neowise_lc = neowise_lc.toPandas()



In [ ]:
#need to convert those magnitudes into flux to be consistent in data structure.
#go to Janskies
#using zeropoints from here: https://wise2.ipac.caltech.edu/docs/release/allsky/expsup/sec4_4h.html
def convert_WISEtoJanskies(mag, magerr, band):
    if band == 'w1':
        zpt = 309.54
    elif band == 'w2':
        zpt = 171.787
            
    flux_Jy = zpt*(10**(-mag/2.5))
    
    #calculate the error
    magupper = mag + magerr
    maglower = mag - magerr
    flux_upper = abs(flux_Jy - (zpt*(10**(-magupper/2.5))))
    flux_lower = abs(flux_Jy - (zpt*(10**(-maglower/2.5))))
    
    fluxerr_Jy = (flux_upper + flux_lower) / 2.0
    
    return flux_Jy, fluxerr_Jy

In [ ]:
def convert_decimal_float(changetype):

    #columns which are themselves arrays
    changetype['mjd'] = [changetype.mjd[i].astype(float) for i in range(len(changetype))]
    changetype['w1mag'] = [changetype.w1pmag[i].astype(float) for i in range(len(changetype))]
    changetype['w2mag'] = [changetype.w2pmag[i].astype(float) for i in range(len(changetype))]
    changetype['w1error'] = [changetype.w1pmagerr[i].astype(float) for i in range(len(changetype))]
    changetype['w2error'] = [changetype.w2pmagerr[i].astype(float) for i in range(len(changetype))]

    return changetype


In [ ]:
#decimals are causing problems when trying to change the units of these columns,
# so change their type to floats which are more universally useable
floats = convert_decimal_float(pd_neowise_lc)

In [ ]:
##ack, w2 errors don't have same lengths as flux arrays!!
#well, we need to exclude w2 for now and figure this out
#Dave thinks it was a problem with null values getting dropped when he made the catalog
#need to re-make the catalog in order for this to get fixed
for i in range(29):
    print(i, floats['w2mag'][i].size, floats['w2error'][i].size)

In [ ]:
#make new columns with fluxes instead of magnitudes for comparison with other bands
w1flux, w1fluxerr = convert_WISEtoJanskies(floats['w1mag'],floats['w1error'] ,'w1')
flux_lc = pd_neowise_lc.assign(w1flux = w1flux, w1fluxerr = w1fluxerr)

#deprecated until we get w2 uncertainties of the same length as magnitudes
#w2flux, w2fluxerr = convert_WISEtoJanskies(floats['w2mag'],floats['w2error'] ,'w2')
#flux_lc = flux_lc.assign(w2flux = w2flux, w2fluxerr = w2fluxerr)


In [ ]:
#Save the resulting (Jy) light curves to the pandas multiindex data frame
#syntax isn't working and getting data type errors to convert the whole dataframe of arrays at once
#instead try iterating over rows in df and making individual data frames which we then concatenate together.
#this is crazy inefficient, but....I can't make it work any other way
pd_neowise_lc.reset_index()
for index, row in flux_lc.iterrows():
    dfw1 = pd.DataFrame(dict(flux=row.w1flux, err=row.w1fluxerr, time=row.mjd, objectid=row.objnum, band='w1')).set_index(["objectid", "band", "time"])

    #then concatenate each individual df together
    #first make sure that df has been defined before
    try:
        df_lc
    except NameError:
        #df_lc doesn't exist (yet)
        df_lc = dfw1
    else:
        #df_lc exists
        df_lc = pd.concat([df_lc, dfw1])#, dfw2])


### 2.4 MAST: Pan-STARRS
Code ideas taken from this website: https://ps1images.stsci.edu/ps1_dr2_api.html

In [ ]:
def ps1cone(ra,dec,radius,table="mean",release="dr1",format="csv",columns=None,
           baseurl="https://catalogs.mast.stsci.edu/api/v0.1/panstarrs", verbose=False,
           **kw):
    """Do a cone search of the PS1 catalog
    
    Parameters
    ----------
    ra (float): (degrees) J2000 Right Ascension
    dec (float): (degrees) J2000 Declination
    radius (float): (degrees) Search radius (<= 0.5 degrees)
    table (string): mean, stack, or detection
    release (string): dr1 or dr2
    format: csv, votable, json
    columns: list of column names to include (None means use defaults)
    baseurl: base URL for the request
    verbose: print info about request
    **kw: other parameters (e.g., 'nDetections.min':2)
    """
    
    data = kw.copy()
    data['ra'] = ra
    data['dec'] = dec
    data['radius'] = radius
    return ps1search(table=table,release=release,format=format,columns=columns,
                    baseurl=baseurl, verbose=verbose, **data)


def ps1search(table="mean",release="dr1",format="csv",columns=None,
           baseurl="https://catalogs.mast.stsci.edu/api/v0.1/panstarrs", verbose=False,
           **kw):
    """Do a general search of the PS1 catalog (possibly without ra/dec/radius)
    
    Parameters
    ----------
    table (string): mean, stack, or detection
    release (string): dr1 or dr2
    format: csv, votable, json
    columns: list of column names to include (None means use defaults)
    baseurl: base URL for the request
    verbose: print info about request
    **kw: other parameters (e.g., 'nDetections.min':2).  Note this is required!
    """
    
    data = kw.copy()
    if not data:
        raise ValueError("You must specify some parameters for search")
    checklegal(table,release)
    if format not in ("csv","votable","json"):
        raise ValueError("Bad value for format")
    url = f"{baseurl}/{release}/{table}.{format}"
    if columns:
        # check that column values are legal
        # create a dictionary to speed this up
        dcols = {}
        for col in ps1metadata(table,release)['name']:
            dcols[col.lower()] = 1
        badcols = []
        for col in columns:
            if col.lower().strip() not in dcols:
                badcols.append(col)
        if badcols:
            raise ValueError('Some columns not found in table: {}'.format(', '.join(badcols)))
        # two different ways to specify a list of column values in the API
        # data['columns'] = columns
        data['columns'] = '[{}]'.format(','.join(columns))

# either get or post works
#    r = requests.post(url, data=data)
    r = requests.get(url, params=data)

    if verbose:
        print(r.url)
    r.raise_for_status()
    if format == "json":
        return r.json()
    else:
        return r.text


def checklegal(table,release):
    """Checks if this combination of table and release is acceptable
    
    Raises a VelueError exception if there is problem
    """
    
    releaselist = ("dr1", "dr2")
    if release not in ("dr1","dr2"):
        raise ValueError("Bad value for release (must be one of {})".format(', '.join(releaselist)))
    if release=="dr1":
        tablelist = ("mean", "stack")
    else:
        tablelist = ("mean", "stack", "detection")
    if table not in tablelist:
        raise ValueError("Bad value for table (for {} must be one of {})".format(release, ", ".join(tablelist)))


def ps1metadata(table="mean",release="dr1",
           baseurl="https://catalogs.mast.stsci.edu/api/v0.1/panstarrs"):
    """Return metadata for the specified catalog and table
    
    Parameters
    ----------
    table (string): mean, stack, or detection
    release (string): dr1 or dr2
    baseurl: base URL for the request
    
    Returns an astropy table with columns name, type, description
    """
    
    checklegal(table,release)
    url = f"{baseurl}/{release}/{table}/metadata"
    r = requests.get(url)
    r.raise_for_status()
    v = r.json()
    # convert to astropy table
    tab = Table(rows=[(x['name'],x['type'],x['description']) for x in v],
               names=('name','type','description'))
    return tab


def addfilter(dtab):
    """Add filter name as column in detection table by translating filterID
    
    This modifies the table in place.  If the 'filter' column already exists,
    the table is returned unchanged.
    """
    if 'filter' not in dtab.colnames:
        # the filterID value goes from 1 to 5 for grizy
        id2filter = np.array(list('grizy'))
        dtab['filter'] = id2filter[(dtab['filterID']-1).data]
    return dtab


In [ ]:
#Do a panstarrs search
radius = 1.0/3600.0 # radius = 1 arcsec
#plt.rcParams.update({'font.size': 14})
#plt.figure(1,(10,10))

        
#for all objects in our catalog
for ccount, coord in enumerate(coords_list):
    #doesn't take SkyCoord
    ra = CLAGN['RA'][ccount]
    dec = CLAGN['DEC'][ccount]

    #see if there is an object in panSTARRS at this location
    results = ps1cone(ra,dec,radius,release='dr2')
    tab = ascii.read(results)
    
    # improve the format
    # column names don't include which filter it is
    for filter in 'grizy':
        col = filter+'MeanPSFMag'
        tab[col].format = ".4f"
        tab[col][tab[col] == -999.0] = np.nan
        
    #in case there is more than one object within 1 arcsec, sort them by match distance
    tab.sort('distance')
    
    #if there is an object at that location
    if len(tab) > 0:   
        #got a live one
        #print( 'for object', ccount + 1, 'there is ',len(tab), 'match in panSTARRS', tab['objID'])

        #take the closest match as the best match
        objid = tab['objID'][0]
        
        #setup to pull light curve info
        dconstraints = {'objID': objid}
        dcolumns = ("""objID,detectID,filterID,obsTime,ra,dec,psfFlux,psfFluxErr,psfMajorFWHM,psfMinorFWHM,
                    psfQfPerfect,apFlux,apFluxErr,infoFlag,infoFlag2,infoFlag3""").split(',')
        # strip blanks and weed out blank and commented-out values
        dcolumns = [x.strip() for x in dcolumns]
        dcolumns = [x for x in dcolumns if x and not x.startswith('#')]


        #get the actual detections and light curve info for this target
        dresults = ps1search(table='detection',release='dr2',columns=dcolumns,**dconstraints)
        
        #sometimes there isn't actually a light curve for the target???
        try:
            ascii.read(dresults)
        except FileNotFoundError:
            print("There is no light curve")
            #no need to store PanSTARRS data for this one
        else:
            #There is a light curve for this target
            
            #fix the column names to include filter names
            dtab = addfilter(ascii.read(dresults))
            dtab.sort('obsTime')

            #here is the light curve mixed from all 5 bands
            t_panstarrs = dtab['obsTime']
            flux_panstarrs = dtab['psfFlux']  # in Jy
            err_panstarrs = dtab['psfFluxErr']
            filtername = dtab['filter']
            
            #put this single object light curves into a pandas multiindex dataframe
            dfsingle = pd.DataFrame(dict(flux=flux_panstarrs, err=err_panstarrs, time=t_panstarrs, objectid=ccount + 1, band=filtername)).set_index(["objectid", "band", "time"])
            
            #then concatenate each individual df together
            #first make sure that df has been defined before
            try:
                df_lc
            except NameError:
                #df_lc doesn't exist (yet)
                df_lc = dfsingle
            else:
                #df_lc exists
                df_lc = pd.concat([df_lc, dfsingle])
                
            


In [ ]:
#pd.set_option('display.max_rows',None)
df_lc
#pd.reset_option('all')

### 2.5 MAST: ATLAS all-sky stellar reference catalog (g, r, i) < 19mag
 -  MAST has this catalog but it is not clear that it has the individual epoch photometry and it is only accessible with casjobs, not through python notebooks.  

 https://archive.stsci.edu/hlsp/atlas-refcat2#section-a737bc3e-2d56-4827-9ab4-838fbf8d67c1
 
 - if we really want to pursue this, we can put in a MAST helpdesk ticket to see if a) they do have the light curves, and b) they could switch the catalog to a searchable with python version.  There are some ways of accessing casjobs through python (<https://github.com/spacetelescope/notebooks/blob/master/notebooks/MAST/HSC/HCV_CASJOBS/HCV_casjobs_demo.ipynb), but apparently not this particular catalog.  
 

### 2.6 MAST: TESS, Kepler and K2
 - use lightKurve to search all 3

In [ ]:
radius = 1.0  #arcseconds

#for all objects
for count, coord in enumerate(coords_list):
    print("working on object", count, coord)
    
    #use lightkurve to search TESS, Kepler and K2
    search_result = lk.search_lightcurve(coord, radius = radius)
    
    #figure out what to do with the results
    if len(search_result) < 1:
        #there is no data in these missions at this location
        print(count, ': no match')
    else:
        #don't know what this looks like because none of these targets has a light curve
        #https://docs.lightkurve.org/tutorials/1-getting-started/searching-for-data-products.html
        #has a tutorial on how to do this
        #might look something like this:
        #lc_collection = search_result[*].download_all()
        print(count, 'got a live one')


### 2.7 MAST: HCV
 - hubble catalog of variables (https://archive.stsci.edu/hlsp/hcv)
 - follow notebook here to know how to search and download light curves https://archive.stsci.edu/hst/hsc/help/HCV/HCV_API_demo.html

In [101]:
hscapiurl = "https://catalogs.mast.stsci.edu/api/v0.1/hsc"

def hcvcone(ra,dec,radius,table="hcvsummary",release="v3",format="csv",magtype="magaper2",
            columns=None, baseurl=hscapiurl, verbose=False,
            **kw):
    """Do a cone search of the HSC catalog (including the HCV)
    
    Parameters
    ----------
    ra (float): (degrees) J2000 Right Ascension
    dec (float): (degrees) J2000 Declination
    radius (float): (degrees) Search radius (<= 0.5 degrees)
    table (string): hcvsummary, hcv, summary, detailed, propermotions, or sourcepositions
    release (string): v3 or v2
    magtype (string): magaper2 or magauto (only applies to summary table)
    format: csv, votable, json
    columns: list of column names to include (None means use defaults)
    baseurl: base URL for the request
    verbose: print info about request
    **kw: other parameters (e.g., 'numimages.gte':2)
    """
    
    data = kw.copy()
    data['ra'] = ra
    data['dec'] = dec
    data['radius'] = radius
    return hcvsearch(table=table,release=release,format=format,magtype=magtype,
                     columns=columns,baseurl=baseurl,verbose=verbose,**data)


def hcvsearch(table="hcvsummary",release="v3",magtype="magaper2",format="csv",
              columns=None, baseurl=hscapiurl, verbose=False,
           **kw):
    """Do a general search of the HSC catalog (possibly without ra/dec/radius)
    
    Parameters
    ----------
    table (string): hcvsummary, hcv, summary, detailed, propermotions, or sourcepositions
    release (string): v3 or v2
    magtype (string): magaper2 or magauto (only applies to summary table)
    format: csv, votable, json
    columns: list of column names to include (None means use defaults)
    baseurl: base URL for the request
    verbose: print info about request
    **kw: other parameters (e.g., 'numimages.gte':2).  Note this is required!
    """
    
    data = kw.copy()
    if not data:
        raise ValueError("You must specify some parameters for search")
    if format not in ("csv","votable","json"):
        raise ValueError("Bad value for format")
    url = "{}.{}".format(cat2url(table,release,magtype,baseurl=baseurl),format)
    if columns:
        # check that column values are legal
        # create a dictionary to speed this up
        dcols = {}
        for col in hcvmetadata(table,release,magtype)['name']:
            dcols[col.lower()] = 1
        badcols = []
        for col in columns:
            if col.lower().strip() not in dcols:
                badcols.append(col)
        if badcols:
            raise ValueError('Some columns not found in table: {}'.format(', '.join(badcols)))
        # two different ways to specify a list of column values in the API
        # data['columns'] = columns
        data['columns'] = '[{}]'.format(','.join(columns))

    # either get or post works
    # r = requests.post(url, data=data)
    r = requests.get(url, params=data)

    if verbose:
        print(r.url)
    r.raise_for_status()
    if format == "json":
        return r.json()
    else:
        return r.text


def hcvmetadata(table="hcvsummary",release="v3",magtype="magaper2",baseurl=hscapiurl):
    """Return metadata for the specified catalog and table
    
    Parameters
    ----------
    table (string): hcvsummary, hcv, summary, detailed, propermotions, or sourcepositions
    release (string): v3 or v2
    magtype (string): magaper2 or magauto (only applies to summary table)
    baseurl: base URL for the request
    
    Returns an astropy table with columns name, type, description
    """
    url = "{}/metadata".format(cat2url(table,release,magtype,baseurl=baseurl))
    r = requests.get(url)
    r.raise_for_status()
    v = r.json()
    # convert to astropy table
    tab = Table(rows=[(x['name'],x['type'],x['description']) for x in v],
               names=('name','type','description'))
    return tab


def cat2url(table="hcvsummary",release="v3",magtype="magaper2",baseurl=hscapiurl):
    """Return URL for the specified catalog and table
    
    Parameters
    ----------
    table (string): hcvsummary, hcv, summary, detailed, propermotions, or sourcepositions
    release (string): v3 or v2
    magtype (string): magaper2 or magauto (only applies to summary table)
    baseurl: base URL for the request
    
    Returns a string with the base URL for this request
    """
    checklegal_hcv(table,release,magtype)
    if table == "summary":
        url = "{baseurl}/{release}/{table}/{magtype}".format(**locals())
    else:
        url = "{baseurl}/{release}/{table}".format(**locals())
    return url


def checklegal_hcv(table,release,magtype):
    """Checks if this combination of table, release and magtype is acceptable
    
    Raises a ValueError exception if there is problem
    """
    
    releaselist = ("v2", "v3")
    if release not in releaselist:
        raise ValueError("Bad value for release (must be one of {})".format(
            ', '.join(releaselist)))
    if release=="v2":
        tablelist = ("summary", "detailed")
    else:
        tablelist = ("summary", "detailed", "propermotions", "sourcepositions",
                    "hcvsummary", "hcv")
    if table not in tablelist:
        raise ValueError("Bad value for table (for {} must be one of {})".format(
            release, ", ".join(tablelist)))
    if table == "summary":
        magtypelist = ("magaper2", "magauto")
        if magtype not in magtypelist:
            raise ValueError("Bad value for magtype (must be one of {})".format(
                ", ".join(magtypelist)))


def resolve(name):
    """Get the RA and Dec for an object using the MAST name resolver
    
    Parameters
    ----------
    name (str): Name of object

    Returns RA, Dec tuple with position
    """

    resolverRequest = {'service':'Mast.Name.Lookup',
                       'params':{'input':name,
                                 'format':'json'
                                },
                      }
    resolvedObjectString = mastQuery(resolverRequest)
    resolvedObject = json.loads(resolvedObjectString)
    # The resolver returns a variety of information about the resolved object, 
    # however for our purposes all we need are the RA and Dec
    try:
        objRa = resolvedObject['resolvedCoordinate'][0]['ra']
        objDec = resolvedObject['resolvedCoordinate'][0]['decl']
    except IndexError as e:
        raise ValueError("Unknown object '{}'".format(name))
    return (objRa, objDec)


def mastQuery(request, url='https://mast.stsci.edu/api/v0/invoke'):
    """Perform a MAST query.

    Parameters
    ----------
    request (dictionary): The MAST request json object
    url (string): The service URL

    Returns the returned data content
    """
    
    # Encoding the request as a json string
    requestString = json.dumps(request)
    r = requests.post(url, data={'request': requestString})
    r.raise_for_status()
    return r.text

def convertACSmagtoflux(date, filterstring, mag, magerr):
    
    #date is currently in MJD and needs to be in ISO Format (YYYY-MM-DD)
    #use astropy to handle this properly
    t = Time(date, format = 'mjd')
    
    #t.iso has this, but also the hrs, min, sec, so need to truncate those
    tiso = t.iso[0:10]
    q = acszpt.Query(date=tiso, detector='WFC', filt=filterstring)
    zpt_table = q.fetch()
    print(zpt_table)
    zpt = zpt_table['VEGAmag'].value
    
    #ACS provides conversion to erg/s/cm^2/angstrom
    flux = 10**((mag - zpt)/(-2.5))  #now in erg/s/cm^2/angstrom
    #calculate the error
    magupper = mag + magerr
    maglower = mag - magerr
    flux_upper = abs(flux - (10**((magupper - zpt)/(-2.5))))
    flux_lower =  abs(flux - (10**((maglower - zpt)/(-2.5))))
    
    fluxerr = (flux_upper + flux_lower) / 2.0

    #now convert from erg/s/cm^2/angstrom to Jy
    #first convert angstrom to Hz using c
    c = 3E8
    flux = flux * c /(1E-10) #now in erg/s/cm^2/Hz
    fluxerr = fluxerr * c /(1E-10) #now in erg/s/cm^2/Hz

    flux = flux * (1E-23) # now in Jy
    
    fluxerr = fluxerr * (1E-23) # now in Jy

    return flux, fluxerr
            
                        

In [ ]:
#Do an HCV search
radius = 1.0/3600.0 # radius = 1 arcsec



for ccount, coord in enumerate(coords_list):

    #doesn't take SkyCoord
    ra = CLAGN['RA'][ccount]
    dec = CLAGN['DEC'][ccount]

    #IC 1613 from the demo
    ra = 16.19913
    dec = 2.11778
    
    #look in the summary table for anything within a radius of our targets
    tab = hcvcone(ra,dec,radius,table="hcvsummary")
    if tab == '':
        print (ccount, 'no matches')
    else:
        print(ccount, 'got a live one')
        
        tab = ascii.read(tab)
                
        matchid = tab['MatchID'][0]  #take the first one, assuming it is the nearest match
        
        #just pulling one filter for an example (more filters are available)
        try:
            src_814 = ascii.read(hcvsearch(table='hcv',MatchID=matchid,Filter='ACS_F814W'))
        except FileNotFoundError:
            #that filter doesn't exist for this target
            print("no F814W filter info for this target")
        else:        
            time_814 = src_814['MJD']
            mag_814 = src_814['CorrMag']  #need to convert this to flux
            magerr_814 = src_814['MagErr']

            filterstring = 'F814W'

            #uggg, ACS has time dependent flux zero points.....
            #going to cheat for now and only use one time, but could imagine this as a loop
            #https://www.stsci.edu/hst/instrumentation/acs/data-analysis/zeropoints
            flux, fluxerr = convertACSmagtoflux(time_814[0], filterstring, mag_814, magerr_814)
            
            #put this single object light curves into a pandas multiindex dataframe
            dfsingle_814 = pd.DataFrame(dict(flux=flux, err=fluxerr, time=time_814, objectid=ccount + 1, band='F814W')).set_index(["objectid", "band", "time"])

            #then concatenate each individual df together
            #first make sure that df has been defined before
            try:
                df_lc
            except NameError:
                #df_lc doesn't exist (yet)
                df_lc = dfsingle_606
            else:
                #df_lc exists
                df_lc = pd.concat([df_lc,  dfsingle_814])


## 3. Find light curves for these targets in relevant, non-NASA catalogs


### Gaia (Faisst)
- astroquery.gaia will presumably work out of the box for this

In [ ]:
############ EXTRACT GAIA DATA FOR OBJECTS ##########

## Select Gaia table (DR3)
Gaia.MAIN_GAIA_TABLE = "gaiaedr3.gaia_source"

## Define search radius
radius = u.Quantity(20, u.arcsec)

## Search and Cross match.
# This can be done in a smarter way by matching catalogs on the Gaia server, or grouping the
# sources and search a larger area.

# get catalog
gaia_table = Table()
t1 = time.time()
for cc,coord in enumerate(coords_list):
    print(len(coords_list)-cc , end=" ")

    gaia_search = Gaia.cone_search_async(coordinate=coord, radius=radius , background=True)
    gaia_search.get_data()["dist"].unit = "deg"
    gaia_search.get_data()["dist"] = gaia_search.get_data()["dist"].to(u.arcsec) # Change distance unit from degrees to arcseconds
    
    
    # match
    if len(gaia_search.get_data()["dist"]) > 0:
        gaia_search.get_data()["input_object_name"] = CLAGN["Object Name"][cc] # add input object name to catalog
        sel_min = np.where( (gaia_search.get_data()["dist"] < 1*u.arcsec) & (gaia_search.get_data()["dist"] == np.nanmin(gaia_search.get_data()["dist"]) ) )[0]
    else:
        sel_min = []
        
    #print("Number of sources matched: {}".format(len(sel_min)) )
    
    if len(sel_min) > 0:
        gaia_table = vstack( [gaia_table , gaia_search.get_data()[sel_min]] )
    else:
        gaia_table = vstack( [gaia_table , gaia_search.get_data()[sel_min]] )

print("\nSearch completed in {:.2f} seconds".format((time.time()-t1) ) )
print("Number of objects mached: {} out of {}.".format(len(gaia_table),len(CLAGN) ) )

In [ ]:
########## EXTRACT PHOTOMETRY #########
# Note that the fluxes are in e/s, not very useful. However, there are magnitudes (what unit??) but without errors.
# We can get the errors from the flux errors?

## Define keys (columns) that will be used later. Also add wavelength in angstroms for each filter
mag_keys = ["phot_bp_mean_mag" , "phot_g_mean_mag" , "phot_rp_mean_mag"]
magerr_keys = ["phot_bp_mean_mag_error" , "phot_g_mean_mag_error" , "phot_rp_mean_mag_error"]
flux_keys = ["phot_bp_mean_flux" , "phot_g_mean_flux" , "phot_rp_mean_flux"]
fluxerr_keys = ["phot_bp_mean_flux_error" , "phot_g_mean_flux_error" , "phot_rp_mean_flux_error"]
mag_lambda = ["5319.90" , "6735.42" , "7992.90"]

## Get photometry. Note that this includes only objects that are 
# matched to the catalog. We have to add the missing ones later.
_phot = gaia_table[mag_keys]
_err = hstack( [ 2.5/np.log(10) * gaia_table[e]/gaia_table[f] for e,f in zip(fluxerr_keys,flux_keys) ] )
gaia_phot2 = hstack( [_phot , _err] )

## Clean up (change units and column names)
_ = [gaia_phot2.rename_column(f,m) for m,f in zip(magerr_keys,fluxerr_keys)]
for key in magerr_keys:
    gaia_phot2[key].unit = "mag"
gaia_phot2["input_object_name"] = gaia_table["input_object_name"].copy()

## Also add object for which we don't have photometry.
# Add Nan for now, need to think about proper format. Also, there are probably smarter ways to do this.
# We do this by matching the object names from the original catalog to the photometry catalog. Then add
# an entry [np.nan, ...] if it does not exist. To make life easier, we add a dummy entry as the first
# row so we can compy all the 
gaia_phot = Table( names=gaia_phot2.keys() , dtype=gaia_phot2.dtype )
for ii in range(len(CLAGN)):
    sel = np.where( CLAGN["Object Name"][ii] == gaia_phot2["input_object_name"] )[0]
    if len(sel) > 0:
        gaia_phot = vstack([gaia_phot , gaia_phot2[sel] ])
    else:
        tmp = Table( np.repeat(np.NaN , len(gaia_phot2.keys())) , names=gaia_phot2.keys() , dtype=gaia_phot2.dtype )
        gaia_phot = vstack([gaia_phot , tmp ])

In [ ]:
gaia_phot.pprint_all()

### ASAS-SN (all sky automated survey for supernovae) has a website that can be manually searched (Faisst)
- see if astroquery.vizier can find it



### icecube has a 2008 - 2018 catalog which we can download and is small (Faisst)
- https://icecube.wisc.edu/data-releases/2021/01/all-sky-point-source-icecube-data-years-2008-2018/

## 4. Make plots of luminosity as a function of time
- model plots after https://arxiv.org/pdf/2111.09391.pdf


In [ ]:
#bands we currently have available are WISE & pan-starrs
availband = ['w1', 'g', 'r', 'i', 'z','y', 'FERMIGTRIG', 'SAXGRBMGRB', 'F814W']
obj = 1  #pick one to look at for now
fig, ax = plt.subplots(figsize=(9,5))

for l in range(len(availband)):
    band_lc = df_lc.loc[(obj, availband[l]), :]
    band_lc.reset_index(inplace=True)
    ax.errorbar(band_lc.time, band_lc.flux, band_lc.err, capsize = 3.0, label = availband[l])
    
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.set_xlabel('Time(MJD)')
ax.set_ylabel('Flux(Jy)')
plt.legend()

## Image extension: look for archival images of these targets
- NASA NAVO use cases should help us to learn how to do this
- can use the cutout service now in astropy from the first fornax use case

## ML Extension 
Consider training a ML model to do light curve classification based on this sample of CLAGN
 - once we figure out which bands these are likely to be observed in, could then have a optical + IR light curve classifier
 - what would the features of the light curve be?
 - what models are reasonable to test as light curve classifiers?
 - could we make also a sample of TDEs, SNe, flaring AGN? - then train the model to distinguish between these things?
 - need a sample of non-flaring light curves
 
After training the model:
 - would then need a sample of optical + IR light curves for "all" galaxies = big data to run the model on.

Some resources to consider:
- https://github.com/dirac-institute/ZTF_Boyajian
- https://ui.adsabs.harvard.edu/abs/2022AJ....164...68S/abstract
- https://ui.adsabs.harvard.edu/abs/2019ApJ...881L...9F/abstract

